In [21]:
#| default_exp preprocessing

# PREPROCESSING

> API details.

In [3]:
#| export
import numpy as np

In [4]:
#| export
def good_luck():
    return 'pass'

In [5]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()